In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6 * 6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
x = [1,2,3,4]
x[1:]

[2, 3, 4]

In [11]:
params = list(net.parameters())
print(len(params))

10


In [18]:
params[1].size(),params[2].size() # params[1]은 뭐임? >> bias

(torch.Size([6]), torch.Size([16, 6, 3, 3]))

In [23]:
params[6].size()

torch.Size([84, 120])

In [27]:
input = torch.randn(3, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0087,  0.1399, -0.1023,  0.0414,  0.0020, -0.1570,  0.0549, -0.1319,
          0.0859, -0.1093],
        [-0.0225,  0.1304, -0.1111,  0.0330,  0.0005, -0.1336,  0.0453, -0.1179,
          0.0775, -0.1263],
        [-0.0259,  0.1163, -0.1051,  0.0516, -0.0172, -0.1427,  0.0215, -0.1375,
          0.0800, -0.1159]], grad_fn=<AddmmBackward>)


In [28]:
net.zero_grad()  # 이건 뭘까#
out.backward(torch.randn(3,10))